In [ ]:

import numpy as np
import random
import random


class Sensor:
    # Settings for robot sensor.
    def __init__(self, grid):
        self.grid = grid

    L = 0.1
    L_s = 0.05
    L_s2 = 0.025

    def sense_location(self):
        """
        Uses sensor to detect current coordinates.
        :return: Tuple of coordinates based on sensing probabilities.
        """
        rand = random.random()
        if rand <= self.L:
            return self.grid.robot_location
        elif rand <= self.L + self.L_s * 8:
            return self.grid.robot_adj_location()
        elif rand <= self.L + self.L_s * 8 + self.L_s2 * 16:
            return self.grid.robot_adj2_location()
        else:
            return None

class Robot:
    def __init__(self, sensor, hmm):
        self.sensor = sensor
        self.x_guess, self.y_guess = 0, 0
        self.hmm = hmm

    def guess_move(self):
        sensed_location = self.sensor.sense_location()
        print("Sensor senses: ", sensed_location)
        self.hmm.forward_step(sensed_location)
        guessed_move, probability = self.hmm.most_probable()
        print("Robot thinks it's in: ", guessed_move, " with probability: ", probability)
        return guessed_move, probability


class Direction:
    NORTH, EAST, SOUTH, WEST = range(4)
    DIRS = [NORTH, EAST, SOUTH, WEST]

    def __init__(self):
        pass

    @classmethod
    def random(cls, exempt_dir=None):
        dirs = [cls.NORTH, cls.EAST, cls.SOUTH, cls.WEST]
        if exempt_dir:
            dirs.remove(exempt_dir)
            return dirs[random.randint(0, 2)]
        else:
            return dirs[random.randint(0, 3)]
        

directionclass = Direction()
class Grid:
    robot = None

    def __init__(self, width = 10 , height = 10 ):
        self.width = width
        self.height = height
        self.robot_location = random.randint(0, width - 1), random.randint(0, height - 1)
        self.robot_dir = Direction.random()

    def robot_adj_location(self):
        """
        Called by sensor to return an adjacent location.
        :return: Tuple containing a random adjacent location to the robot, or None if that particular location
                 is out of bounds.
        """
        x, y = self.robot_location

        l_s = [(x - 1, y - 1), (x - 1, y), (x - 1, y + 1), (x, y - 1), (x, y + 1), (x + 1, y - 1), (x + 1, y),
               (x + 1, y + 1)]

        adj = l_s[random.randint(0, 7)]

        # wall check
        if adj[0] >= self.width or adj[1] >= self.height or adj[0] < 0 or adj[1] < 0:
            return None
        else:
            return adj

    def robot_adj2_location(self):
        """
        Called by sensor to return an adjacent location 2 spaces away.
        :return: Tuple containing a random adjacent location 2 away from the robot, or None if that particular location
                 is out of bounds.
        """
        x, y = self.robot_location

        ls_2 = [(x - 2, y - 2), (x - 2, y - 1), (x - 2, y), (x - 2, y + 1), (x - 2, y + 2), (x - 1, y - 2),
                (x - 1, y + 2), (x, y - 2), (x, y + 2), (x + 1, y - 2), (x + 1, y + 2), (x + 2, y - 2), (x + 2, y - 1),
                (x + 2, y), (x + 2, y + 1), (x + 2, y + 2)]

        adj = ls_2[random.randint(0, 15)]

        # wall check
        if adj[0] >= self.width or adj[1] >= self.height or adj[0] < 0 or adj[1] < 0:
            return None
        else:
            return adj

    def robot_faces_wall(self):
        """
        Called by sensor to determine is robot faces wall.
        :return: Boolean whether or not robot is facing a wall.
        """
        x, y = self.robot_location

        # NORTH, EAST, SOUTH, WEST
        next_locations = [(x, y + 1), (x + 1, y), (x, y - 1), (x - 1, y)]
        next_coord = next_locations[self.robot_dir]
        if next_coord[0] >= self.width or next_coord[1] >= self.height or next_coord[0] < 0 or next_coord[1] < 0:
            return True
        else:
            return False

    def move_robot(self):
        """
        Move robot according to strategy.
        """
        # 30% Chance to change direction.
        rand = random.random()
        if rand <= 0.3:
            self.robot_dir = Direction.random(self.robot_dir)
        # Changes direction until robot doesn't face wall.
        while self.robot_faces_wall():
            self.robot_dir = Direction.random(self.robot_dir)

        x, y = self.robot_location

        # Moves forward in robot's direction.
        # NORTH, EAST, SOUTH, WEST
        next_locations = [(x, y + 1), (x + 1, y), (x, y - 1), (x - 1, y)]
        self.robot_location = next_locations[self.robot_dir]
        



directionclass2 = Direction()
class HMM:
    def __init__(self, width = 10, height = 10 ):
        self.width = width
        self.height = height
        self.t_matrix = self.create_t_matrix()
        self.none_matrix = self.none_matrix()
        self.f_matrix = self.create_priors()

    def create_priors(self):
        length = self.width * self.height * 4
        priors = [float(1) / length] * length
        return np.array(priors)

    def create_t_matrix(self):
        width = self.width
        height = self.height
        t = np.array(np.zeros(shape=(width * height * 4, width * height * 4)))

        for i in range(width * height * 4):
            x = int(i // (height * 4))
            y = int((i // 4) % height)
            heading = i % 4
            prev_states = self.probable_transitions((x, y, heading))
            for (xcoord, ycoord, direction), probability in prev_states:
                t[i, int(xcoord * height * 4 + ycoord * 4 + direction)] = probability
        return t

    def probable_transitions(self, state):
        """
        Finds the neighbors of particular coord.
        :param state: tuple of (x, y, heading)
        :return : list of empty squares that are adjacent to, with probability
        """
        x, y, direction = state
        # came from: NORTH, EAST, SOUTH, WEST
        neighbors = [(x, y - 1), (x - 1, y), (x, y + 1), (x + 1, y)]
        prev_square = neighbors[direction]
        prev_x, prev_y = prev_square

        # Check bounds
        if prev_x < 0 or prev_x >= self.width or prev_y < 0 or prev_y >= self.height:
            return []

        # Always 0.7 chance if coming in same direction.
        square_dir = [((prev_x, prev_y, direction), 0.7)]
        dirs_left = list(Direction.DIRS)
        dirs_left.remove(direction)
        # Check if any directions point to walls.
        faces_wall = []
        if Direction.WEST in dirs_left:
            if prev_x == 0:
                faces_wall.append((prev_x, prev_y, Direction.WEST))
            else:
                square_dir.append(((prev_x, prev_y, Direction.WEST), 0.1))
        if Direction.EAST in dirs_left:
            if prev_x == self.width - 1:
                faces_wall.append((prev_x, prev_y, Direction.EAST))
            else:
                square_dir.append(((prev_x, prev_y, Direction.EAST), 0.1))
        if Direction.SOUTH in dirs_left:
            if prev_y == 0:
                faces_wall.append((prev_x, prev_y, Direction.SOUTH))
            else:
                square_dir.append(((prev_x, prev_y, Direction.SOUTH), 0.1))
        if Direction.NORTH in dirs_left:
            if prev_y == self.height - 1:
                faces_wall.append((prev_x, prev_y, Direction.NORTH))
            else:
                square_dir.append(((prev_x, prev_y, Direction.NORTH), 0.1))

        for state in faces_wall:
            square_dir.append((state, float(1) / (4 - len(faces_wall))))
        return square_dir

    def assign_adjacents(self, o, possible_adj2, probability):
        for poss_x, poss_y in possible_adj2:
            index = poss_x * self.height * 4 + poss_y * 4
            for i in range(4):
                o[index + i, index + i] = probability

    def create_sensor_matrix(self, sensed_coord):
        if sensed_coord is None:
            return self.none_matrix
        width = self.width
        height = self.height
        o = np.array(np.zeros(shape=(width * height * 4, width * height * 4)))
        x, y = sensed_coord

        # Assign probability of 0.1 for sensed_coord
        index = x * height * 4 + y * 4
        for i in range(4):
            o[index + i, index + i] = 0.1

        # Assign probability of 0.05 for directly adjacent squares
        self.assign_adjacents(o, self.possible_adj(x, y), 0.05)
        # Assign probability of 0.025 for directly adjacent squares
        self.assign_adjacents(o, self.possible_adj2(x, y), 0.025)

        return o

    def none_matrix(self):
        width = self.width
        height = self.height
        o = np.array(np.zeros(shape=(width * height * 4, width * height * 4)))
        for i in range(width * height * 4):
            x = i / (height * 4)
            y = (i / 4) % height

            num_adj = 8 - len(self.possible_adj(x, y))
            num_adj2 = 16 - len(self.possible_adj2(x, y))

            o[i, i] = 0.1 + 0.05 * num_adj + 0.025 * num_adj2
        return o

    def possible_adj(self, x, y):
        possible_adj = [(x - 1, y - 1), (x - 1, y), (x - 1, y + 1), (x, y - 1), (x, y + 1), (x + 1, y - 1), (x + 1, y),
                        (x + 1, y + 1)]

        # Bound check
        for poss_x, poss_y in list(possible_adj):
            if poss_x >= self.width or poss_x < 0 or poss_y >= self.height or poss_y < 0:
                possible_adj.remove((poss_x, poss_y))

        return possible_adj

    def possible_adj2(self, x, y):
        possible_adj2 = [(x - 2, y - 2), (x - 2, y - 1), (x - 2, y), (x - 2, y + 1), (x - 2, y + 2), (x - 1, y - 2),
                         (x - 1, y + 2), (x, y - 2), (x, y + 2), (x + 1, y - 2), (x + 1, y + 2), (x + 2, y - 2),
                         (x + 2, y - 1),
                         (x + 2, y), (x + 2, y + 1), (x + 2, y + 2)]

        # Bound check
        for poss_x, poss_y in list(possible_adj2):
            if poss_x >= self.width or poss_x < 0 or poss_y >= self.height or poss_y < 0:
                possible_adj2.remove((poss_x, poss_y))

        return possible_adj2

    def forward_step(self, coord):
        f = self.f_matrix
        o = self.create_sensor_matrix(coord)
        t = self.t_matrix

        f = t.dot(f).dot(o)
        f /= np.sum(f)

        self.f_matrix = f

    def most_probable(self):
        f = self.f_matrix
        max_prob_idx = np.argmax(f)
        x = int(max_prob_idx // (self.height * 4))
        y = int((max_prob_idx // 4) % self.height)
        return (x, y), f[max_prob_idx]


from time import sleep




def start_robot(width = 10 , height = 10):
    """
    Sets up the world and robot and enters endless loop of guessing.
    :param size:
    """
    # create grid that contains world state
    grid = Grid(width, height)
    # create sensor, which queries the grid
    sensor = Sensor(grid)
    # create robot, which guesses location based on sensor
    hmm = HMM(width, height)
    robot = Robot(sensor, hmm)

    moves = 0
    guessed_right = 0
    while True:
        # move robot
        grid.move_robot()
        moves += 1
        print ("\nRobot is in: ", grid.robot_location)
        guessed_move, probability = robot.guess_move()
        if guessed_move == grid.robot_location:
            guessed_right += 1
        man_distance = abs(guessed_move[0] - grid.robot_location[0]) + abs(guessed_move[1] - grid.robot_location[1])
        print ("Manhattan distance: ", man_distance)
        print ("Robot has been correct:", float(guessed_right) / moves, "of the time.")
        sleep(1)

if __name__ == '__main__':
    """
    The main function called when main.py is run from the command line.

    Example usage:
    > python main.py --width 10 --height 10
    First argument defines the width of the grid, second, the height.
    """
    # handle arguments
    # import argparse
    # 
    # parser = argparse.ArgumentParser()
    # parser.add_argument("--width", type=int, required=True)
    # parser.add_argument("--height", type=int, required=True)
    # args = parser.parse_args()
    # if args.width <= 1 or args.height <= 1:
    #     raise argparse.ArgumentTypeError("Values must be greater than one.")
    #
    start_robot()